In [1]:
from astropy.wcs import WCS
from astropy.io import fits
from astropy import units as u
from astropy.table import Table
from astropy.time import Time
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion,PixCoord, PointSkyRegion, PointPixelRegion, LineSkyRegion, LinePixelRegion
%matplotlib qt
allrun = Table.read('Basel_8_runaways_all.tsv', format='ascii.ecsv')[:5]
allrun_coord_now = SkyCoord(ra=allrun['RA_ICRS_1'], 
                        dec=allrun['DE_ICRS_1'],
                        distance=allrun['rgeo'], 
                        pm_ra_cosdec=allrun['rmRA'],
                        pm_dec=allrun['rmDE'],
                        obstime=Time('J2000')+500*u.kyr)

allrun_coord_earlier = allrun_coord_now.apply_space_motion(dt=-100*u.kyr)


with fits.open('./Basel_8_extra10pc.fits') as fits_file:
    image = fits_file[0]
    wcs = WCS(image.header)
    fig, ax = plt.subplots(subplot_kw={'projection': wcs}, figsize=(8, 8))
    ax.imshow(image.data, 
              cmap='gray')
    ax.set_xlabel('Right Ascension (hms)')
    ax.set_ylabel('Declination (degrees)')

    c = SkyCoord("98.5582 +8.0381", distance=1456, unit=(u.deg ,u.deg, u.pc))
    radius = Angle(0.3, 'deg')
    region = CircleSkyRegion(c,radius)
    region_pix = region.to_pixel(wcs)
    region_pix.plot(ax=ax, 
                    color='red', 
                    lw=2)
    
    start_skys = allrun_coord_now
    end_skys = allrun_coord_earlier

    region_sky = [LineSkyRegion(start=start_sky, end=end_sky) for start_sky,end_sky in zip(allrun_coord_now,allrun_coord_earlier)]
    region_sky_pix = [region_sky_.to_pixel(wcs) for region_sky_ in region_sky]
    for c in allrun_coord_now:
        point_sky = PointSkyRegion(center=c)
        point_pix = point_sky.to_pixel(wcs)
        point_pix.plot(ax=ax,
                       color='yellow')
    for region_pix in region_sky_pix:
        region_pix.plot(ax=ax,
                        color='cyan')

QSocketNotifier: Can only be used with threads started with QThread


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xb5 in position 11368: invalid start byte

In [13]:
from astropy.wcs import WCS
from astropy.io import fits
from astropy import units as u
from astropy.table import Table,join
from astropy.time import Time
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion,PixCoord, PointSkyRegion, PointPixelRegion, LineSkyRegion, LinePixelRegion
from typing import List
class ownCluster:
    def __init__(self, cluster_name,coordinates: SkyCoord, diameter: Angle,add_members: List[int]=[]):
        self.name = cluster_name
        self.coordinates = coordinates
        self.diameter = diameter
snr_center = SkyCoord('05 26 30  +42 56 00', distance=3000*u.pc,unit=(u.hourangle, u.deg))
vro = ownCluster('VRO', snr_center,0.5*u.deg)
vro.coordinates

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    (81.625, 42.93333333, 3000.)>

In [14]:
from astroquery.skyview import SkyView
images = SkyView.get_images(position=vro.coordinates,
                            survey='DSS',
                            radius=1.2*vro.diameter,
                            pixels=1000)
# Extract the WCS information
wcs = WCS(images[0][0].header)
hdu = fits.PrimaryHDU(data=images[0][0].data, header=images[0][0].header)
hdulist = fits.HDUList([hdu])

# Save the fits file
hdulist.writeto('vro.fits', overwrite=True)

In [15]:
with fits.open('./vro.fits') as fits_file:
    image = fits_file[0]
    wcs = WCS(image.header)
    fig, ax = plt.subplots(subplot_kw={'projection': wcs}, figsize=(8, 8))
    ax.imshow(image.data, 
              cmap='gray')
    ax.set_xlabel('Right Ascension (hms)')
    ax.set_ylabel('Declination (degrees)')

In [16]:
from astroquery.vizier import Vizier

stars_fromDR3 = Vizier(columns=["*","+_r"],row_limit = -1).query_region(vro.coordinates, 
                                                                        radius=vro.diameter/2, 
                                                                        catalog= 'I/355/gaiadr3',
                                                                        )[0]

stars_fromDR3_dis = Vizier(columns=["*","+_r"],row_limit = -1).query_region(vro.coordinates, 
                                                                        radius=vro.diameter/2, 
                                                                        catalog= 'I/352/gedr3dis',
                                                                        )[0]

stars_in_region = join(stars_fromDR3,stars_fromDR3_dis,keys='Source',join_type='inner')
mask1 = stars_in_region['rgeo']<4000.42857143 *u.pc
mask2 = stars_in_region['rgeo']>2000.33333333 *u.pc
mask3 = stars_in_region['Gmag']<17*u.mag
stars_in_region = stars_in_region[mask1 & mask2 & mask3]
# stars_in_region = stars_in_region[mask3]

In [17]:
stars_in_region

_r_1,RA_ICRS_1,DE_ICRS_1,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,_r_2,RA_ICRS_2,DE_ICRS_2,rgeo,b_rgeo,B_rgeo,rpgeo,b_rpgeo,B_rpgeo,Flag
,deg,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,,deg,deg,pc,pc,pc,pc,pc,pc,
float64,float64,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,str12,str15,str19,int32,str10,str16,str17,str21,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32
0.240853,81.40192935146,42.75652926718,195254543335518720,0.0253,0.0153,0.2834,0.0310,3.604,0.743,0.038,-3.526,0.025,0.982,19939.00267,4.895,14.938108,7141,13.6,15.704210,1.834e+04,18.71,14.089321,1.614889,-10.48,2.88,--,13.738337,0,0,0,1,1,0,0,0,0,1,0,--,--,--,--,--,--,159300814020178338,--,--,,URAT1-664154587,,--,NCAD002678,,05253646+4245235,,81.40192485437,42.75654493896,0.240853,81.40192935146,42.75652926718,3204.88037000,2919.69263000,3589.39380000,3067.48779000,2759.56006000,3368.58105000,10033
0.215630,81.39519952927,42.79869944025,195255642847144320,0.0235,0.0153,0.2475,0.0302,2.140,0.072,0.037,-2.139,0.026,0.938,17191.64730,4.247,15.099073,9021,17.39,15.450365,1.182e+04,10.09,14.566565,0.883800,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,7883.0,3.9440,-0.8227,2678.6990,1.3786,--,159350813951989064,--,--,,URAT1-664154566,J052534.84+424755.4,1011679,NCAD003103,,05253484+4247551,,81.39519909067,42.79870894621,0.215630,81.39519952927,42.79869944025,3484.86133000,3096.06152000,3959.30273000,3374.84326000,3135.29297000,3675.57861000,10033
0.223208,81.38016426801,42.80059731170,195267393877666560,0.0581,0.0376,0.4252,0.0704,5.248,3.633,0.081,-3.788,0.059,1.013,3277.97296,1.806,16.898354,1456,8.391,17.430850,2578,7.976,16.219770,1.211079,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,5360.7,4.7241,-1.4344,1026.1637,0.7218,--,159360813801601341,--,--,,URAT1-665154458,J052531.22+424802.1,--,NCAD003133,,05253124+4248020,,81.38014226405,42.80061414661,0.223208,81.38016426801,42.80059731170,2226.36523000,1919.29724000,2567.78955000,2413.90088000,2004.26392000,2817.39673000,10033
0.223548,81.36979802174,42.81089194766,195267428237404672,0.0609,0.0402,0.3054,0.0773,1.801,0.941,0.092,-1.536,0.066,1.024,3141.10254,1.766,16.944662,1379,6.24,17.489445,2536,7.852,16.237518,1.251926,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,5459.3,4.3985,-2.2412,1608.6954,0.9130,--,159370813697963693,--,--,,URAT1-665154418,,--,NCAD003240,,05252875+4248391,,81.36979231871,42.81089877318,0.223548,81.36979802174,42.81089194766,3895.24707000,2778.69604000,5660.65771000,2776.75879000,2363.95386000,3271.61768000,10033
0.205773,81.38719580638,42.82390684357,195267458297353856,0.0482,0.0314,0.4597,0.0610,2.600,0.845,0.072,-2.459,0.052,1.055,4829.28531,2.166,16.477660,2164,10.6,17.000483,3887,9.341,15.773865,1.226618,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,6118.2,4.4712,-1.4545,1346.8794,1.2584,--,159380813871969304,--,--,,URAT1-665154474,J052532.94+424926.2,--,NCAD003354,,05253293+4249259,,81.38719068418,42.82391777369,0.205773,81.38719580638,42.82390684357,2115.77246000,1867.14758000,2525.75586000,2091.82544000,1878.01990000,2364.19214000,10033
0.247330,81.33018634394,42.81296057840,195267600035207296,0.0296,0.0176,0.4542,0.0364,1.104,-0.861,0.038,-0.691,0.026,0.940,16825.71897,4.067,15.122433,7659,15.21,15.628067,1.321e+04,13.55,14.445837,1.182230,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,6677.9,4.1841,-0.6833,1

In [18]:
source1 = 195633320791780608
source2 = 195633325090480896
m1 = stars_in_region['Source'] == source1
m2 = stars_in_region['Source'] == source2
twostars = stars_in_region[m1 | m2]
twostars

_r_1,RA_ICRS_1,DE_ICRS_1,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000,_r_2,RA_ICRS_2,DE_ICRS_2,rgeo,b_rgeo,B_rgeo,rpgeo,b_rpgeo,B_rpgeo,Flag
,deg,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg,,deg,deg,pc,pc,pc,pc,pc,pc,
float64,float64,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,str12,str15,str19,int32,str10,str16,str17,str21,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int32
0.003716,81.62187230463,42.93624922588,195633320791780608,0.0429,0.0280,0.2980,0.0489,3.977,0.846,0.059,-3.886,0.045,1.021,6276.76299,2.736,16.193027,3013,13.36,16.641203,4690,11.55,15.569920,1.071283,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,6518.7,4.3925,-1.0023,1647.5944,1.0941,--,159520816218474167,--,--,,URAT1-665155263,,--,NC96004733,,05262924+4256105,,81.62186716715,42.93626649576,0.003716,81.62187230463,42.93624922588,3241.66699000,2832.63281000,3757.81396000,2957.65674000,2600.03027000,3453.99927000,10033
0.006438,81.62564389768,42.93974983019,195633325090480896,0.0447,0.0294,0.2922,0.0544,2.950,1.408,0.064,-2.592,0.046,1.046,5882.52580,3.53,16.263456,3004,17.82,16.644405,4535,16.2,15.606478,1.037928,--,--,--,--,0,0,0,1,0,0,0,0,1,1,0,6047.5,4.4779,-1.0638,1386.9924,0.7165,--,159520816256148351,--,--,,URAT1-665155279,,--,NC96004824,,05263015+4256230,,81.62563534911,42.93976135030,0.006438,81.62564389768,42.93974983019,3138.43115000,2643.20166000,3779.71533000,2937.71899000,2546.00830000,3432.86401000,10033


In [19]:
from astropy.wcs import WCS
from astropy.io import fits
from astropy import units as u
from astropy.table import Table
from astropy.time import Time
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion,PixCoord, PointSkyRegion, PointPixelRegion, LineSkyRegion, LinePixelRegion
#%matplotlib qt
allrun = stars_in_region
dt = -20*u.kyr
allrun_coord_now = SkyCoord(ra=allrun['RA_ICRS_1'], 
                        dec=allrun['DE_ICRS_1'],
                        distance=allrun['rgeo'], 
                        pm_ra_cosdec=allrun['pmRA'],
                        pm_dec=allrun['pmDE'],
                        obstime=Time('J2000')+500*u.kyr)
twostars_coord = SkyCoord(ra=twostars['RA_ICRS_1'], 
                        dec=twostars['DE_ICRS_1'],
                        distance=twostars['rgeo'], 
                        pm_ra_cosdec=twostars['pmRA'],
                        pm_dec=twostars['pmDE'],
                        obstime=Time('J2000')+500*u.kyr)
allrun_coord_earlier = allrun_coord_now.apply_space_motion(dt=dt)


with fits.open('./vro.fits') as fits_file:
    image = fits_file[0]
    wcs = WCS(image.header)
    fig, ax = plt.subplots(subplot_kw={'projection': wcs}, figsize=(12, 12))
    ax.imshow(image.data, 
              cmap='gray')
    ax.set_xlabel('Right Ascension (hms)')
    ax.set_ylabel('Declination (degrees)')

    c = SkyCoord("05 26 30  +42 56 00", distance=3000, unit=(u.hourangle ,u.deg, u.pc))
    radius = Angle(vro.diameter/2)
    region = CircleSkyRegion(c,radius)
    region_pix = region.to_pixel(wcs)
    region_pix.plot(ax=ax, 
                    color='red', 
                    lw=2)
    
    start_skys = allrun_coord_now
    end_skys = allrun_coord_earlier

    region_sky = [LineSkyRegion(start=start_sky, end=end_sky) for start_sky,end_sky in zip(allrun_coord_now,allrun_coord_earlier)]
    region_sky_pix = [region_sky_.to_pixel(wcs) for region_sky_ in region_sky]
    for c in allrun_coord_now:
        point_sky = PointSkyRegion(center=c)
        point_pix = point_sky.to_pixel(wcs)
        point_pix.plot(ax=ax,
                       color='green')
    for region_pix in region_sky_pix:
        region_pix.plot(ax=ax,
                        color='cyan')
    for c in twostars_coord:
        point_sky = PointSkyRegion(center=c)
        point_pix = point_sky.to_pixel(wcs)
        point_pix.plot(ax=ax,
                       color='yellow')

c:\Users\Surodeep\AppData\Local\Programs\Python\Python310\lib\site-packages\erfa\core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),


In [20]:
from regions import Regions
points = [PointSkyRegion(center=c) for c in allrun_coord_now]
region_sky.append(region)
region_sky = region_sky+points
allregions = Regions(region_sky)
allregions.write('vro.reg',overwrite=True)
